In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page
from nuwinter.utils import deduplicate_df
from nuwinter.paths import get_pdf_path
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner







/Users/robertstein/anaconda3/envs/nuwinter_env/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
base_dir = Path.home().joinpath("Data/winter")

# name = "IC240412A"

name = "S250119cv"
# name="IC241006A"

prob_threshold = 0.95
n_days = 7

In [3]:
outpath = str(get_pdf_path(name))#.replace("combined", "combined_ztf")
outpath

'/Users/robertstein/Data/nuwinter/S250119cv/winter_candidate_pdf/combined.pdf'

In [4]:
nu = NeutrinoScanner(name) if "IC" in name else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days
)

100%|███████████████████████████████████████| 19/19 [00:00<00:00, 179082.64it/s]


In [5]:
nights = [str(nu.t_min + (i - 1) * u.day).split("T")[0].replace("-", "") for i in range(n_days + 1)]

all_res = []

for night in sorted(nights):
    
    avro_dir = base_dir / f"{night}/avro"

    print(f"Looking for avros in {avro_dir}")
    
    avro_files = list(avro_dir.glob("*.avro"))
    
    for path in avro_files:
        all_res.append(load_avro(path))

if len(all_res) == 0:
    raise FileNotFoundError(f"No avro files found for any night, please download these first using the download notebook!")

res_winter = pd.concat(all_res)

Looking for avros in /Users/robertstein/Data/winter/20250118/avro
Looking for avros in /Users/robertstein/Data/winter/20250119/avro
Looking for avros in /Users/robertstein/Data/winter/20250120/avro
Looking for avros in /Users/robertstein/Data/winter/20250121/avro
Looking for avros in /Users/robertstein/Data/winter/20250122/avro
Looking for avros in /Users/robertstein/Data/winter/20250123/avro
Looking for avros in /Users/robertstein/Data/winter/20250124/avro
Looking for avros in /Users/robertstein/Data/winter/20250125/avro


FileNotFoundError: No avro files found for any night, please download these first using the download notebook!

In [ ]:
plt.hist(res_winter["dec"])

In [ ]:
in_contour = []

for _, row in res_winter.iterrows():
    in_contour.append(nu.in_contour(row["ra"], row["dec"]))

print(f"{np.sum(in_contour)} / {len(in_contour)} candidates in contour")

res_winter = res_winter[in_contour]

In [ ]:
# plt.hist(res_winter[~np.array(in_contour)]["dec"])

In [ ]:
res_winter

In [ ]:
len(res_winter["objectid"])

In [ ]:
# res_winter["match"] = match
# res_winter_match = res_winter[pd.notnull(match)]
res_winter_match = res_winter

In [ ]:
sgscore_cut = 0.7
dist_cut_arcsec = 3.

star_cut = (pd.notnull(res_winter_match["distpsnr1"]) & (res_winter_match["distpsnr1"] < dist_cut_arcsec) & pd.notnull(res_winter_match["sgscore1"]) & (res_winter_match["sgscore1"] > sgscore_cut))

# masks = [
#     res_winter_match["ndethist"] > 0,
#     ~star_cut,
#     res_winter_match["nbad"] < 2,
#     # res_winter_match["chipsf"] < 3.0,
#     # res_winter_match["sumrat"] > 0.7,
#     res_winter_match["fwhm"] < 10.0,
#     # res_winter_match["magdiff"] < 1.6,
#     # res_winter_match["magdiff"] > -1.0,
#     res_winter_match["mindtoedge"] > 100.0,
#     res_winter_match["isdiffpos"],
# ]

masks = [
    res_winter_match["ndethist"] > 0,
    ~star_cut,
    res_winter_match["nbad"] < 2,
    res_winter_match["chipsf"] < 3.0,
    res_winter_match["sumrat"] > 0.7,
    res_winter_match["fwhm"] < 10.0,
    res_winter_match["magdiff"] < 1.6,
    res_winter_match["magdiff"] > -1.0,
    res_winter_match["mindtoedge"] > 100.0,
    res_winter_match["isdiffpos"],
]

mask = np.ones(len(res_winter_match), dtype=bool)
for new_mask in masks:
    mask *= new_mask
    
print(f"{np.sum(mask)} / {len(mask)} detections pass all cuts")
cut2 = res_winter_match[mask]

if len(cut2) > 0:
    cut2 = deduplicate_df(cut2)
    cut2.sort_values(by="objectid", inplace=True)
    
    outpath = str(get_pdf_path(name)).replace("combined", "single")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=False)
            pdf.savefig()
            plt.close()

In [ ]:
list(res_winter_match.columns)